#### XGBoost with Optuna Hyperparameter Tuning

#### Overview
This notebook implements a machine learning pipeline to predict NBA game outcomes using XGBoost with automated hyperparameter optimization via Optuna.

#### Methodology

##### 1. Data Preparation
- Load NBA game dataset from CSV
- Filter games from October 19, 2021 onwards
- Remove non-predictive features (date, team names)
- Separate features (X) and target variable (winning_team)

##### 2. Hyperparameter Optimization
- Use Optuna framework for Bayesian optimization
- Search space includes:
  - Tree structure: `n_estimators`, `max_depth`, `min_child_weight`
  - Learning dynamics: `learning_rate`, `subsample`, `colsample_bytree`
  - Regularization: `gamma`, `reg_alpha`, `reg_lambda`
- 500 trials with 5-fold cross-validation
- Maximize accuracy metric

##### 3. Model Training & Evaluation
- Train final XGBoost classifier with optimal parameters
- Evaluate on held-out test set (20% split)
- Visualize optimization history and hyperparameter importance

#### Expected Outputs
1. Best hyperparameters and cross-validation score
2. Final model test accuracy
3. Optimization history plot
4. Hyperparameter importance visualization

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import optuna
import matplotlib.pyplot as plt

# XGBoost classifier for probability prediction
from xgboost import XGBClassifier

# Scikit-learn utilities
from sklearn.model_selection import cross_val_score
from sklearn.metrics import (
    accuracy_score,                 # Overall prediction accuracy
    classification_report,          # Precision, recall, F1 per class
    roc_auc_score,                  # Discrimination power
    log_loss,                       # Negative log likelihood (calibration metric)
    brier_score_loss,               # Squared error of predicted probabilities
)
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

# Optuna tools for hyperparameter optimization
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler

# Suppress verbose Optuna logging for cleaner output
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Load historical NBA game dataset
print("Loading Data...")
df: pd.DataFrame = pd.read_csv("../data/csv/dataset.csv")
print("Data Loaded Successfully!")

# Convert the 'date' column to datetime objects for time-based splits
df["date"] = pd.to_datetime(df["date"], errors="coerce")

# Training data: all games before October 1, 2024
print("\nSplitting data by season...")
train_df: pd.DataFrame = df[df["date"] < "2024-10-01"].reset_index(drop=True)

# Test data: games from 2024-25 season (unseen during training)
test_df: pd.DataFrame = df[df["date"] >= "2024-10-01"].reset_index(drop=True)

print(f"Training set: {len(train_df)} games")
print(f"Test set: {len(test_df)} games")

# Columns not used for model training
drop_cols: list[str] = ["date", "home_team", "away_team"]

# Remove non-feature columns
train_df: pd.DataFrame = train_df.drop(drop_cols, axis=1)
test_df: pd.DataFrame = test_df.drop(drop_cols, axis=1)

# Separate features (X) and target (y)
X_train: pd.DataFrame = train_df.drop("winning_team", axis=1)
y_train: pd.Series = train_df["winning_team"]
X_test: pd.DataFrame = test_df.drop("winning_team", axis=1)
y_test: pd.Series = test_df["winning_team"]

# Check class imbalance to guide XGBoost's scale_pos_weight
imbalance_ratio: float = (y_train == 0).sum() / (y_train == 1).sum()
print(f"\nClass imbalance ratio (Home:Away) ≈ {imbalance_ratio:.2f}")

# Define the Optuna Objective function on which the optimization will happen
def objective(trial) -> float:
    """
    Defines the objective function for Optuna hyperparameter optimization.
    Optimizes negative log loss (probability calibration) for NBA game outcomes.
    
    Parameters:
        trial: Optuna trial object for suggesting hyperparameter values.

    Returns:
        Mean 5-fold cross-validated negative log loss.
    """
    params: dict[str, int | float | str] = {
        "n_estimators": trial.suggest_int("n_estimators", 300, 900),
        "max_depth": trial.suggest_int("max_depth", 3, 8),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.15, log=True),
        "subsample": trial.suggest_float("subsample", 0.7, 0.95),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 0.95),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-4, 10, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.01, 20, log=True),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 15),
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 0.5, 5.0),

        # Fixed parameters
        "objective": "binary:logistic",     # Predict probabilities for binary outcome
        "eval_metric": "logloss",           # Optimize log loss
        "random_state": 42,                 # Reproducibility
        "n_jobs": -1,                       # Use all CPU cores
    }

    model: XGBClassifier = XGBClassifier(**params)

    # 5-fold cross-validation on **real data** (no resampling)
    # Scoring uses negative log loss because Optuna maximizes the objective
    score: float = cross_val_score(
        model,
        X_train,
        y_train,
        scoring="neg_log_loss",
        cv=5,
        n_jobs=-1,
    )

    # Return mean CV score
    return float(np.mean(score))


# Run Optuna tuning
print("\nStarting Optuna tuning...\n")
study = optuna.create_study(
    direction="maximize",                                           # Maximize neg_log_loss = minimize log loss
    sampler=TPESampler(n_startup_trials=20, seed=42),               # Bayesian TPE sampler
    pruner=MedianPruner(n_startup_trials=10, n_warmup_steps=20),    # Early stopping for unpromising trials
)
study.optimize(
    objective,
    n_trials=400,               # Maximum number of trials
    timeout=10800,              # 3 hours max
    n_jobs=-1,                  # Parallel evaluation
    show_progress_bar=True,
)

# Training the final model
print("\nBEST TRIAL:")
print(f"Best CV Log Loss: {-study.best_trial.value:.4f}")
best_params = study.best_trial.params.copy()
best_params.update(
    {
        "objective": "binary:logistic",
        "eval_metric": "logloss",
        "random_state": 42,
        "n_jobs": -1,
    }
)
base_model: XGBClassifier = XGBClassifier(**best_params)
base_model.fit(X_train, y_train)

# Calibrate the probabilities
print("\nCalibrating probabilities...")
calibrated_model: CalibratedClassifierCV = CalibratedClassifierCV(
    base_model,
    method="isotonic",  # best for larger datasets
    cv=5,
)
calibrated_model.fit(X_train, y_train)

# Evaluate the calibrated model on the test set selected at the start of the pipeline
y_pred: list[int] = calibrated_model.predict(X_test)
y_pred_proba: list[tuple[float, float]] = calibrated_model.predict_proba(X_test)[:, 1]
test_accuracy: float = accuracy_score(y_test, y_pred)
test_roc_auc: float = roc_auc_score(y_test, y_pred_proba)
test_log_loss: float = log_loss(y_test, y_pred_proba)
test_brier: float = brier_score_loss(y_test, y_pred_proba)

# Calibration analysis
print("\nTEST SET RESULTS (2024-25):")
print(f"Accuracy: {test_accuracy*100:.2f}%")
print(f"ROC-AUC: {test_roc_auc:.4f}")
print(f"Log Loss: {test_log_loss:.4f}")
print(f"Brier Score: {test_brier:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nCalibration by bins:")
prob_bins: list[float] = [0, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
for i in range(len(prob_bins) - 1):
    mask = (y_pred_proba >= prob_bins[i]) & (y_pred_proba < prob_bins[i + 1])
    if mask.sum() > 0:
        actual_rate = y_test[mask].mean()
        print(
            f"Predicted {prob_bins[i]:.1f}-{prob_bins[i+1]:.1f}: "
            f"Actual away win rate = {actual_rate:.2%} (n={mask.sum()})"
        )

# Visualize the calibration plot to identify the strenght and weakneses
plt.style.use("seaborn-v0_8-darkgrid")

# Calibration Curve
fraction_of_positives, mean_predicted_value = calibration_curve(
    y_test, y_pred_proba, n_bins=10, strategy="uniform"
)

plt.figure(figsize=(10, 8))
plt.plot(mean_predicted_value, fraction_of_positives, "s-",
         label=f"XGBoost (Brier: {test_brier:.4f})")
plt.plot([0, 1], [0, 1], "k--", label="Perfect Calibration")
plt.xlabel("Mean Predicted Probability")
plt.ylabel("Fraction of Positives")
plt.title("Calibration Curve")
plt.legend()
plt.grid(True)
plt.show()

print("\nPipeline completed successfully!")